# Candidate Shaplet
We will explore the results from the candidate shaplet extraction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

Pull in the data from the shaplet script

In [2]:
df = pd.read_csv('../../shaplets/140_candidate_shaplets.csv', header=0, index_col=0)

For now we will just sum the results from the search.  'sum' represents the sum of the top 10% of euclidean distances for that particular shaplet 

In [3]:
df['sum'] = df.sum(1)

In [4]:
df['sum'].describe()

count     796.000000
mean     3685.670797
std       264.934928
min      2999.698280
25%      3500.255702
50%      3677.293737
75%      3870.237682
max      4753.168935
Name: sum, dtype: float64

Use the shaplet with the smalled sum as the candidate shaplet

In [5]:
shaplet_index = df['sum'].sort_values().index[0]

This value was discovered by printing out the value from the euclidean distance script.  
***In the future, we should be storing the shaplet length in the file*** 

In [6]:
shaplet_len = 97.5

Pull in data from walking and running from the same user

In [7]:
walking = pd.read_csv('../../data/140/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)
running = pd.read_csv('../../data/140/17_treadmill_6mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)
medium_walking = pd.read_csv('../../data/140/15_treadmill_4mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)
laundry = pd.read_csv('../../data/140/19_seated_folding_laundry.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)

In [8]:
import numpy as np

Extract the candidate shaplet.

In [9]:
shaplet = walking.loc[shaplet_index:shaplet_index+shaplet_len].reset_index()

## Testing the Shaplet 
For the same user, we will test the candidate shaplet against walking and running from the same user.
To simplify this test, we will only test values from [7000:13000] with the intention of doing this in real time in the future.  

For each index in that range, a shaplet will be extracted of the walking shaplet length and tested using euclidean distance against the candidate shaplet

In [10]:
walk_test = []
run_test = []
laundry_test = []
med_test = []
for x in range(7000, 13000):
    wslice = walking.loc[x:x+shaplet_len].reset_index()
    rslice = running.loc[x:x+shaplet_len].reset_index()
    lslice = laundry.loc[x:x+shaplet_len].reset_index()
    mslice = medium_walking.loc[x:x+shaplet_len].reset_index()
    walk_test.append(np.linalg.norm(shaplet['x'] - wslice['x']))
    run_test.append(np.linalg.norm(shaplet['x'] - rslice['x']))
    laundry_test.append(np.linalg.norm(shaplet['x'] - lslice['x']))
    med_test.append(np.linalg.norm(shaplet['x'] - mslice['x']))

In [11]:
walk_test = pd.Series(walk_test)
run_test = pd.Series(run_test)
laundry_test = pd.Series(laundry_test)
med_test = pd.Series(med_test)

In [25]:
plt.close()
plt.plot(walk_test)
#plt.plot(med_test, 'r')
#plt.plot(laundry_test, 'r')
#plt.plot(run_test, 'k')

In [24]:
shaplet_models = df['sum'].sort_values().index[0:2]
plt.close()
for model in shaplet_models:
    pd_model = walking.loc[model:model+shaplet_len].reset_index()
    plt.plot(pd_model['x'])